In [4]:
import os
import pandas as pd

In [5]:
def load_csv_as_df(file_name, sub_directories, column_numbers=None, column_names=None):
    '''
    Load any csv as a pandas dataframe. Provide the filename, the subdirectories, and columns to read(if desired).
    '''
    base_path = os.getcwd()
    full_path = base_path + sub_directories + file_name

    if column_numbers is not None:
        df = pd.read_csv(full_path, usecols=column_numbers)
    else:
        df = pd.read_csv(full_path)

    if column_names is not None:
        df.columns = column_names

    return df

In [6]:
def load_data(file_name, sub_directories, with_routes=False):
    if not with_routes:
        col_names = ['taxi_id', 'time', 'longitude', 'latitude',  'occupancy_status', 'speed']
    else:
        col_names += ['route_number', 'route_start', 'route_end']

    df = load_csv_as_df(file_name, sub_directories, None, col_names)

    return df

In [7]:
df = load_data('TaxiData.csv', '/')

In [8]:
def filter_data_by_gps(df, min_lat, max_lat, min_long, max_long, with_pass=False):
    """
    :param df: data-frame with latitude and longitude columns
    :param min_lat: minimum latitude value
    :param max_lat: max latitude value
    :param min_long: min longitude
    :param max_long: max longitude
    :param with_pass: required to have a passenger (occupancy_status column)
    :return: data-frame
    """

    all_taxi_ids = df['taxi_id'].unique()
    print('There are ', len(all_taxi_ids), ' taxi ids in this dataset!')

    near_lat = df[(df['latitude'] >= min_lat) & (df['latitude'] <= max_lat)] # 0.0203
    print('There are ', len(near_lat), ' GPS readings in your latitude range!')

    near_lat_and_long = near_lat[(near_lat['longitude'] >= min_long) & (near_lat['longitude'] <= max_long)] #0.022334

    print('There are ', len(near_lat_and_long), ' GPS readings in your latitude and longitude range!')
    taxi_ids = near_lat_and_long['taxi_id'].unique()
    print('There are ', len(taxi_ids), ' taxi ids near the airport!')

    if with_pass:
        with_pass = near_lat_and_long[near_lat_and_long['occupancy_status'] == 1]
        print('There are ', len(with_pass), ' GPS readings in your latitude and longitude range with a passenger!')
        with_pass_ids = with_pass['taxi_id'].unique()
        print('There are ', len(with_pass_ids), ' taxi ids in your latitude and longitude range with a passenger!')
        return with_pass
    else:
        return near_lat_and_long

In [9]:
# Shenzhen Train station west GPS 22.5316, 113.903 !@#$%^&* WRONG!

# Correct GPS = 22.534578, 114.110521

train_station_west_gps = (22.534578, 114.110521)
lat_diff = 0.025
long_diff = 0.025

near_west_train_df = filter_data_by_gps(df,
                                        train_station_west_gps[0] - lat_diff,
                                        train_station_west_gps[0] + lat_diff,
                                        train_station_west_gps[1] - long_diff,
                                        train_station_west_gps[1] + long_diff,
                                        with_pass=True)

There are  14728  taxi ids in this dataset!
There are  20527691  GPS readings in your latitude range!
There are  5259769  GPS readings in your latitude and longitude range!
There are  10126  taxi ids near the airport!
There are  2840110  GPS readings in your latitude and longitude range with a passenger!
There are  7998  taxi ids in your latitude and longitude range with a passenger!


In [10]:
near_west_train_df.head()

,taxi_id,time,longitude,latitude,occupancy_status,speed
18,22223,13:09:34,114.125999,22.542633,1,39
25,22223,11:42:11,114.117897,22.552767,1,0
26,22223,11:45:54,114.117218,22.546766,1,33
28,22223,12:08:43,114.116837,22.550283,1,21
33,22223,11:47:24,114.103165,22.544300,1,47


In [11]:
# North Train Station in Shenzhen is 22.605502, 114.023724 and 22.613580, 114.034568.
train_station_north_gps_min = (22.605502, 114.023724)
train_station_north_gps_max = (22.613580, 114.034568)


near_north_train_df = filter_data_by_gps(df,
                                         train_station_north_gps_min[0],
                                         train_station_north_gps_max[0],
                                         train_station_north_gps_min[1],
                                         train_station_north_gps_max[1],
                                         with_pass=True)


There are  14728  taxi ids in this dataset!
There are  1587117  GPS readings in your latitude range!
There are  135172  GPS readings in your latitude and longitude range!
There are  6873  taxi ids near the airport!
There are  46030  GPS readings in your latitude and longitude range with a passenger!
There are  4665  taxi ids in your latitude and longitude range with a passenger!


In [12]:
near_north_train_df.head()

,taxi_id,time,longitude,latitude,occupancy_status,speed
6958,22225,19:25:31,114.027283,22.610550,1,0
7056,22225,13:48:46,114.025215,22.610018,1,54
7136,22225,08:09:13,114.028847,22.613300,1,0
7215,22225,13:48:52,114.024452,22.609583,1,57
7216,22225,08:09:21,114.029266,22.612667,1,2


In [13]:
west_taxi_ids = near_west_train_df['taxi_id'].unique()
north_taxi_ids = near_north_train_df['taxi_id'].unique()

intersection_taxi_ids = list(set(west_taxi_ids) & set(north_taxi_ids))
print(len(intersection_taxi_ids))

3540


In [14]:
def get_gps_records_with_taxi_id_in(taxi_id_list, df):
    return df[df['taxi_id'].isin(taxi_id_list)]

In [15]:
relevant_gps_records = get_gps_records_with_taxi_id_in(intersection_taxi_ids, df)
print(len(relevant_gps_records))

13247420


In [16]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date: pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [17]:
def label_trajectories(df):
    df['time'] = lookup(df['time'])
    updated_dfs = []
    taxi_ids = df['taxi_id'].unique()
    print('There are ', len(taxi_ids), ' in this data')
    empty_route = -1
    trajectory_number = 1

    completed_count = 0

    for taxi_id in taxi_ids:
        # get the df for that taxis
        taxi_df = df.loc[df['taxi_id'] == taxi_id]
        taxi_df.sort_values(by=['time'], inplace=True)
        passenger_got_in = False
        route_numbers = []
        route_starts = []
        route_ends = []

        for index, row in taxi_df.iterrows():
            passenger_in_taxi = row['occupancy_status']

            # Do we already have a passenger?
            if passenger_got_in:
                if passenger_in_taxi:
                    # trajectory still going
                    route_starts.append(False)
                    route_ends.append(False)
                    route_numbers.append(trajectory_number)
                    continue
                elif not passenger_in_taxi:
                    # trajectory ended
                    passenger_got_in = False
                    route_starts.append(False)
                    route_ends.append(True)
                    route_numbers.append(trajectory_number)
                    trajectory_number += 1

            elif passenger_in_taxi:
                passenger_got_in = True
                route_starts.append(True)
                route_ends.append(False)
                route_numbers.append(trajectory_number)

            else:
                route_starts.append(False)
                route_ends.append(False)
                route_numbers.append(empty_route)

        taxi_df['route_number'] = route_numbers
        taxi_df['route_start'] = route_starts
        taxi_df['route_end'] = route_ends
        updated_dfs.append(taxi_df)
        completed_count += 1

        if completed_count % 100 == 0:
            print('Completed ', completed_count, ' taxi_ids out of ', len(taxi_ids))

    return pd.concat(updated_dfs)

In [18]:
%%time
labeled_relevant_gps_records_df = label_trajectories(relevant_gps_records)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


There are  3540  in this data


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

Completed  100  taxi_ids out of  3540
Completed  200  taxi_ids out of  3540
Completed  300  taxi_ids out of  3540
Completed  400  taxi_ids out of  3540
Completed  500  taxi_ids out of  3540
Completed  600  taxi_ids out of  3540
Completed  700  taxi_ids out of  3540
Completed  800  taxi_ids out of  3540
Completed  900  taxi_ids out of  3540
Completed  1000  taxi_ids out of  3540
Completed  1100  taxi_ids out of  3540
Completed  1200  taxi_ids out of  3540
Completed  1300  taxi_ids out of  3540
Completed  1400  taxi_ids out of  3540
Completed  1500  taxi_ids out of  3540
Completed  1600  taxi_ids out of  3540
Completed  1700  taxi_ids out of  3540
Completed  1800  taxi_ids out of  3540
Completed  1900  taxi_ids out of  3540
Completed  2000  taxi_ids out of  3540
Completed  2100  taxi_ids out of  3540
Completed  2200  taxi_ids out of  3540
Completed  2300  taxi_ids out of  3540
Completed  2400  taxi_ids out of  3540
Completed  2500  taxi_ids out of  3540
Completed  2600  taxi_ids out of  

In [19]:
labeled_relevant_gps_records_df.to_csv('original-labeled-train-train-routes.csv', encoding='utf-8', index=False)

In [20]:
labeled = labeled_relevant_gps_records_df

In [21]:
labeled = labeled[labeled['route_number'] != -1]
labeled["route_number"] = labeled["route_number"] + 500000 
labeled.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,taxi_id,time,longitude,latitude,occupancy_status,speed,route_number,route_start,route_end
14915,22228,2018-12-08 00:00:10,113.988853,22.536200,1,42,500001,True,False
10845,22228,2018-12-08 00:00:40,113.984467,22.536383,1,69,500001,False,False
13495,22228,2018-12-08 00:00:54,113.982170,22.536467,1,58,500001,False,False
13677,22228,2018-12-08 00:01:10,113.979202,22.536516,1,76,500001,False,False
12673,22228,2018-12-08 00:01:24,113.976646,22.536850,1,68,500001,False,False


In [22]:
labeled_starts = labeled[labeled['route_start'] == True]
print(len(labeled_starts))

labeled_ends = labeled[labeled['route_end'] == True]
print(len(labeled_ends))

185169
182855


In [23]:
north_train_starts = filter_data_by_gps(labeled_starts,
                                         train_station_north_gps_min[0],
                                         train_station_north_gps_max[0],
                                         train_station_north_gps_min[1],
                                         train_station_north_gps_max[1],
                                         with_pass=False)
print(len(north_train_starts))

There are  3540  taxi ids in this dataset!
There are  5736  GPS readings in your latitude range!
There are  1622  GPS readings in your latitude and longitude range!
There are  1347  taxi ids near the airport!
1622


In [24]:
west_train_starts = filter_data_by_gps(labeled_starts,
                                        train_station_west_gps[0] - lat_diff,
                                        train_station_west_gps[0] + lat_diff,
                                        train_station_west_gps[1] - long_diff,
                                        train_station_west_gps[1] + long_diff,
                                        with_pass=False)
print(len(west_train_starts))

There are  3540  taxi ids in this dataset!
There are  127524  GPS readings in your latitude range!
There are  38995  GPS readings in your latitude and longitude range!
There are  3429  taxi ids near the airport!
38995


In [25]:
north_train_ends = filter_data_by_gps(labeled_ends,
                                      train_station_north_gps_min[0],
                                      train_station_north_gps_max[0],
                                      train_station_north_gps_min[1],
                                      train_station_north_gps_max[1],
                                      with_pass=False)
print(len(north_train_ends))

There are  3480  taxi ids in this dataset!
There are  5802  GPS readings in your latitude range!
There are  1971  GPS readings in your latitude and longitude range!
There are  1569  taxi ids near the airport!
1971


In [26]:
west_train_ends = filter_data_by_gps(labeled_ends,
                                     train_station_west_gps[0] - lat_diff,
                                     train_station_west_gps[0] + lat_diff,
                                     train_station_west_gps[1] - long_diff,
                                     train_station_west_gps[1] + long_diff,
                                     with_pass=False)
print(len(west_train_ends))

There are  3480  taxi ids in this dataset!
There are  120001  GPS readings in your latitude range!
There are  35819  GPS readings in your latitude and longitude range!
There are  3433  taxi ids near the airport!
35819


In [27]:
def find_column_intersection(df1, df2, col_name):
    """
    :param df1: data-frame
    :param df2: data-frame
    :param col_name: name of column
    :return: list of values in both columns
    """

    col_one_unique = df1[col_name].unique()
    col_two_unique = df2[col_name].unique()
    intersection = list(set(col_one_unique) & set(col_two_unique))
    
    return intersection

In [28]:
north_to_west_train_route_numbers = find_column_intersection(north_train_starts, west_train_ends, 'route_number')
print(len(north_to_west_train_route_numbers))

208


In [29]:
west_to_north_train_route_numbers = find_column_intersection(west_train_starts, north_train_ends, 'route_number')
print(len(west_to_north_train_route_numbers))

312


In [30]:
def get_rows_with_col_value_in(df, val_list, col_name):
    return df[df[col_name].isin(val_list)]

In [31]:
north_to_west_route_df = get_rows_with_col_value_in(labeled, 
                                                    north_to_west_train_route_numbers, 
                                                    'route_number')

In [32]:
west_to_north_route_df = get_rows_with_col_value_in(labeled, 
                                                    west_to_north_train_route_numbers, 
                                                    'route_number')

In [33]:
north_to_west_route_df.to_csv('north-to-west-routes-no-grids.csv', encoding='utf-8', index=False)
west_to_north_route_df.to_csv('west-to-north-routes-no-grids.csv', encoding='utf-8', index=False)